In [1]:
pwd

'C:\\Users\\kathy\\Desktop\\Python\\Ext_Finance_Data\\Code'

In [2]:
# Test the full import out
# Need to make the APRA date variable more dynamic
# Then need to add visualisations
# What is the proper way to link everything together?

In [3]:
# APRA Monthly Statistics

# Household Deposits (Cash held by Australians in Banks)

# https://www.apra.gov.au/monthly-authorised-deposit-taking-institution-statistics back series Mar 2019 -

# Loans to households: Housing: Owner-occupied
# Loans to households: Housing: Investment
# Deposits by households

In [4]:
# File Name
# Monthly authorised deposit-taking institution statistics back-series March 2019 - November 2021
# Still need to add query to automate this

In [5]:
#############################################################

# Section: Libraries

#############################################################

import pandas as pd # Data Analysis Library
import numpy as np
import matplotlib.pyplot as plt # Data Visualisation Library
import matplotlib.ticker as ticker
%matplotlib inline

import seaborn as sns # Data Visualisation Library
import requests # For downloading 
import datetime
from datetime import datetime
import re # Regex
import numbers 

import pickle # for saving/loading files

pd.options.mode.chained_assignment = None  # default='warn'

In [6]:
#############################################################

# Section: Functions

#############################################################

# pickle_save: save the files after importing and reading them
def pickle_save(name, to_save):
    with open('../Data/' + name + '.pickle', 'wb') as handle:
        pickle.dump(to_save, handle, protocol=pickle.HIGHEST_PROTOCOL)
# picle_load: load previously saved files
def pickle_load(name):
    with open('../Data/' + name + '.pickle', 'rb') as handle:
        load_data = pickle.load(handle)
    return load_data

# match: search each string element within a list ('list_search') in a string ('in_string') and  
# return the match. Used to define the type of variable within the ABS Lending Indicator datasets.
def match(list_search, in_string):
    # need to add restrictions on input types to list + string
    result = [f for f in list_search if re.search(f, desclist)] 
    return(result)

# human_format: format numbers to be more readable 
def human_format(num):
    num = float('{:.3g}'.format(num))
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    return '{}{}'.format('{:f}'.format(num).rstrip('0').rstrip('.'), ['', 'K', 'M', 'B', 'T'][magnitude])

In [7]:
apra_end_date = 'November 2021'
apra_end_date_dt = datetime.strptime(apra_end_date, "%B %Y")

In [8]:
file_dict = {}  
file_dict = {
    
    'APRA' : {
        'ADIs - Total' : {
            'Metadata' : {
                'dl_name': 'Monthly authorised deposit-taking institution statistics back-series March 2019 - November 2021',
                'series':  ['Loans to households: Housing: Owner-occupied',
                           'Loans to households: Housing: Investment', 'Deposits by households']
            }
        }
    }
}

for source in file_dict:
    if source == 'RBA':
        for key in file_dict[source]:
            file_dict[source][key]['Metadata']['row_drop'] = ['Description', 'Frequency', 'Type', 'Units', 'Source', 
                                                              'Publication date', 'Series ID']
            file_dict[source][key]['Metadata']['dl_url'] = 'https://www.rba.gov.au/statistics/tables/xls/'
            file_dict[source][key]['Metadata']['sheet_name'] = 'Data'
            file_dict[source][key]['Metadata']['filetype'] = '.xlsx'
            file_dict[source][key]['Metadata']['ID_row'] = 'Series ID'
            file_dict[source][key]['Metadata']['unit_row'] = 'Units'
            file_dict[source][key]['Metadata']['publication_row'] = 'Publication date'
            file_dict[source][key]['Metadata']['skiprow'] = 1

    elif source == 'ABS':
        for key in file_dict[source]:
            file_dict[source][key]['Metadata']['row_drop'] = ['Unit', 'Series Type', 'Data Type', 'Frequency', 
                                                              'Collection Month', 'No. Obs', 'Series Start', 
                                                              'Series End', 'Series ID']
            file_dict[source][key]['Metadata']['dl_url'] = 'https://www.abs.gov.au/statistics/economy/finance/lending-indicators/latest-release/'
            file_dict[source][key]['Metadata']['sheet_name'] = 'Data1'
            file_dict[source][key]['Metadata']['filetype'] = '.xls'
            file_dict[source][key]['Metadata']['ID_row'] = 'Series ID'
            file_dict[source][key]['Metadata']['unit_row'] = 'Unit'
            file_dict[source][key]['Metadata']['publication_row'] = 'Series End'
            file_dict[source][key]['Metadata']['skiprow'] = 0
            
    elif source == 'APRA':
        for key in file_dict[source]:
            file_dict[source][key]['Metadata']['row_drop'] = ['NONE']
            file_dict[source][key]['Metadata']['dl_url'] = 'https://www.apra.gov.au/sites/default/files/2022-01/' # Needs to be dynamic as well
            file_dict[source][key]['Metadata']['sheet_name'] = 'Table 1'
            file_dict[source][key]['Metadata']['filetype'] = '.xlsx'
            file_dict[source][key]['Metadata']['ID_row'] = 'Manual'
            file_dict[source][key]['Metadata']['unit_row'] = 'Manual'     # No units. Need to add one manually
            file_dict[source][key]['Metadata']['publication_row'] = 'Manual'    # Need to add one manually
            file_dict[source][key]['Metadata']['skiprow'] = 1    

In [9]:
# Download all the external files from the file_dict dictionary
for source in file_dict:
    for key in file_dict[source]:
        dl_name = file_dict[source][key]['Metadata']['dl_name'] + file_dict[source][key]['Metadata']['filetype']
        dl_url = file_dict[source][key]['Metadata']['dl_url'] + dl_name
        r = requests.get(dl_url)
        with open("../Input/" + dl_name, 'wb') as f:
                  f.write(r.content)

FileNotFoundError: [Errno 2] No such file or directory: './Input/Monthly authorised deposit-taking institution statistics back-series March 2019 - November 2021.xlsx'

In [ ]:
# Need to adjust the import step for APRA which is different from ABS/ RBA

In [ ]:
# Copy the below onto step 0

In [ ]:
# External files are imported and cleaned.Each external file is saved into the dictionary 
# ('import_file_dict') with a lookup which details the unique series ID, description and units.

for source in file_dict:
    
    for key in file_dict[source]:   
        
        data_name = file_dict[source][key]['Metadata']['dl_name']
        row_filter = file_dict[source][key]['Metadata']['row_drop']
        series_filter = file_dict[source][key]['Metadata']['series']
        series_name = file_dict[source][key]['Metadata']['ID_row']

        read_path = './Input/' + data_name + file_dict[source][key]['Metadata']['filetype']
        df = pd.read_excel(read_path, sheet_name = file_dict[source][key]['Metadata']['sheet_name'], 
                           skiprows = file_dict[source][key]['Metadata']['skiprow'])
        df = df.rename(columns={ df.columns[0]: 'Title' })

        description = df.columns.values
        
        if series_name == 'Manual':
            series_id_row = description
        else:
            series_id_row = df[df['Title'] == series_name].values.tolist()[0]
            series_id_row[0] = 'Title'

        df.columns = series_id_row

        # Extract Publication Date
        
        if file_dict[source][key]['Metadata']['publication_row'] == 'Manual':
            pub_date = apra_end_date_dt
        else: 
            pub_date = df.loc[df['Title'] == file_dict[source][key]['Metadata']['publication_row']].values[0,1]
            if type(pub_date) == datetime.datetime:
                pub_date = pub_date.strftime("%d-%b-%Y")
                
        print(file_dict[source][key]['Metadata']['dl_name'], 'latest publication date:', pub_date) #loc = index check

        # Transform Data 
        #if series_name == 'Manual':
        #    series_id = description
        #else:
        #    series_id = df[df['Title'] == file_dict[source][key]['Metadata']['ID_row']].values[0]
        
        if source == 'APRA':
            units = ['$ million'] * len(description)
        else:
            units = df[df['Title'] == file_dict[source][key]['Metadata']['unit_row']].values[0] # Unit values
        
        #series_lookup = pd.DataFrame(list(zip(series_id,description,units)), columns=['Series ID','Description','Unit'])
        series_lookup = pd.DataFrame(list(zip(series_id_row,description,units)), columns=['Series ID','Description','Unit'])
        series_lookup = series_lookup[series_lookup['Series ID'] != 'Series ID'] # Need to save these in the dictionary output
        series_lookup = series_lookup.loc[series_lookup['Series ID'].isin(series_filter), :]

        # select the series_to_filter
        column_filter = series_lookup.loc[:,'Series ID'].values.tolist()
        column_filter.insert(0, 'Title')

        # Drop Rows
        if source == 'APRA':
            df_fmt = df.copy()
        else:
            df_fmt = df[~df['Title'].isin(row_filter)]
            df_fmt = df_fmt.dropna(subset=['Title']) # Remove rows in first column with NA

        for i in range(len(units)):
            if(df_fmt.iloc[:,i].dtype == np.float64 or df_fmt.iloc[:,i].dtype == np.int64):
                
                if source == 'RBA':
                    if units[i].strip() == """'000""":
                        df_fmt.iloc[:,i] = df_fmt.iloc[:,i] * 1000
                    elif units[i].strip() == '$ million':
                        df_fmt.iloc[:,i] = df_fmt.iloc[:,i] * 1000000
                elif source == 'ABS':
                    if units[i].strip() == '$ Millions':
                        df_fmt.iloc[:,i] = df_fmt.iloc[:,i] * 1000000
                elif source == 'APRA':
                    if units[i].strip() == '$ million':
                        df_fmt.iloc[:,i] = df_fmt.iloc[:,i] * 1000000
                    
        # Drop columns
        df_fmt = df_fmt.loc[:,column_filter] # Keep relevant series

        df_fmt = df_fmt.reset_index(drop=True)
        df_fmt = df_fmt.rename(columns={'Title': 'Date'})
        df_fmt = df_fmt.convert_dtypes() # Convert variable types
        
        file_dict[source][key]['Import_Data'] = df_fmt 
        file_dict[source][key]['Lookup'] = series_lookup

In [ ]:
#############################################################

# Section: Format Tables

#############################################################

# variable_dict: match terms used to define the variables

variable_dict = {
    
    'RBA' : {
        'measure_type' : {
            'list' : ['number of accounts', 'total number of transactions', 'total value of transactions', 
                      'balances accruing interest', 'total balances', 'credit limits'],
            'alias' : {
                'number of accounts' : 'accounts',
                'total number of transactions' : 'transactions',
                'total value of transactions' : 'transaction value',
                'balances accruing interest' : 'interest balance',
                'total balances' : 'balance',
                'credit limits' : 'limits'
            }
        }
    },
    'APRA' : {
        'measure_type' : {
            'list' : ['Loans to households: Housing: Owner-occupied', 'Loans to households: Housing: Investment', 
                      'Deposits by households'],
            'alias' : {
                'Loans to households: Housing: Owner-occupied' : 'Mortgages OO',
                'Loans to households: Housing: Investment' : 'Mortgages Inv',
                'Deposits by households' : 'Deposits'
            }
        }
    }
}

In [ ]:
df_fmt.head()

In [ ]:
searchlist.str.lower()

In [ ]:
searchlist2 = [x.lower() for x in searchlist]

In [ ]:
searchlist2

In [ ]:
desclist

In [ ]:
desclist

In [ ]:
# table_structure_dict: contains the variable structure for each table

table_structure_dict = {}

for source in file_dict:
    table_structure_dict[source] = {}

    for key in file_dict[source]:   
        
        table_structure_dict[source][key] = {}

        lookup = file_dict[source][key]['Lookup']
        
        file_ref = file_dict[source][key]['Metadata']['dl_name']

        for seriesloop in lookup['Series ID'].tolist():

            table_structure_dict[source][key][seriesloop] = {}
            
            table_structure_dict[source][key][seriesloop]['file_ref'] = file_ref

            if source == 'APRA':
                desclist = lookup.loc[lookup['Series ID'] == seriesloop, 'Description'].values[0]
            else:
                desclist = lookup.loc[lookup['Series ID'] == seriesloop, 'Description'].values[0].lower() # why lower?

            for var in variable_dict[source]:
                
                searchlist = variable_dict[source][var]['list']
                
                if match(searchlist, desclist) == []:
                    if var == 'measure_type':
                        output = 'total housing excluding refinancing'
                    else:
                        output = 'all'
                else: 
                    output = match(searchlist, desclist)[0]
                
                if 'alias' in variable_dict[source][var].keys():
                    table_structure_dict[source][key][seriesloop][var] = variable_dict[source][var]['alias'][output]
                else:
                    table_structure_dict[source][key][seriesloop][var] = output

In [ ]:
#############################################################

# Section: Aggregate imported tables to final state

#############################################################

# Final_table_dict: contains the final aggregated dataframes that are used for visualisation/ analysis
final_table_dict = {}

for source in table_structure_dict:
    j = 0
    
    final_table_dict[source] = {}
    
    # ABS data is combined into a master table with a single measure column
    if source == 'ABS':  
        
        # Initalise empty list to store a dataframe per file
        df_master_list = [None] * len(table_structure_dict[source])

        for key in table_structure_dict[source]:

            df = file_dict[source][key]['Import_Data']
            df = df[df['Date'] >= '2019-06-01']
            df = df.melt(id_vars=["Date"])
            
            # Initalise empty list to store a dataframe per series
            df_list = [None] * len(table_structure_dict[source][key]) 
            
            i = 0

            for series in table_structure_dict[source][key]:
                
                df_loop = df.loc[df['variable']==series,]
                #df_loop['filename'] = final_table_dict[source][key]['Metadata']['dl_name']

                for n in table_structure_dict[source][key][series]:
                    df_loop.loc[:, n] = table_structure_dict[source][key][series][n]

                    if n == 'variable':
                        df_loop.loc[:, table_structure_dict[source][key][series][n]] = df_loop['value']
                    else:  
                        df_loop.loc[:, n] = table_structure_dict[source][key][series][n]

                    df_loop = df_loop.rename(columns={'All':'total housing excluding refinancing'})
                
                df_list[i] = df_loop

                i = i + 1

            df_master_list[j] = pd.concat(df_list)

            j = j + 1

        final_table_dict[source]['Final_Data'] = pd.concat(df_master_list)
                                              
    # Data in RBA CC data has multiple columns
    elif source == 'RBA':     
        
        for key in table_structure_dict[source]:
            df = file_dict[source][key]['Import_Data']
            
            if key == 'Credit Card Data - Australia':
                source_fmt = 'RBA-Credit'
            elif key == 'Debit Card Data - Australia': 
                source_fmt = 'RBA-Debit'
            
            for series in table_structure_dict[source][key]:
                
                df = df.rename({series:table_structure_dict[source][key][series]['measure_type']}, axis=1)

            final_table_dict[source_fmt] = {}                     
            final_table_dict[source_fmt]['Final_Data'] = df 
            
    elif source == 'APRA':
        
        for key in table_structure_dict[source]:
            df = file_dict[source][key]['Import_Data']
            
            if key == 'ADIs - Total':
                source_fmt = 'APRA-Monthly'
            
            for series in table_structure_dict[source][key]:
                
                df = df.rename({series:table_structure_dict[source][key][series]['measure_type']}, axis=1)

            final_table_dict[source_fmt] = {}                     
            final_table_dict[source_fmt]['Final_Data'] = df 

In [ ]:
# Above is put back into zero